In [1]:
# Importing the libraries
import numpy as np 
import pandas as pd
import glob, os

In [2]:
## TRAINING DATA

path = './'                     # use your path
all_files = glob.glob(os.path.join(path, "data*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

dataset = pd.DataFrame()
list_ = []
for file_ in all_files:
    df = pd.read_csv(file_,index_col=None, header=0)
    print('Size:', df.shape)
    list_.append(df.as_matrix())

comb_np_array = np.vstack(list_)
dataset = pd.DataFrame(comb_np_array)

('Size:', (30327, 2053))


/usr/lib/python2.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (30327, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))
('Size:', (91989, 2053))


In [3]:
print('Size:', dataset.shape)

('Size:', (2446320, 2053))


In [4]:
del comb_np_array
del df
del list_

In [5]:
nf=5 # number of frame for each image

(len_data,b)=(dataset.shape)

image_row=nf*2
image_col=513


In [6]:
#(len_data,b)=(dataset.shape)
inputdim=b-1;


X = dataset.iloc[:, 0:inputdim].values
y  = dataset.iloc[:, inputdim].values
#X_train = dataset.iloc[0:len_train, 0:inputdim].values
#y_train  = dataset.iloc[0:len_train, inputdim].values
#X_test = dataset.iloc[len_train:len_train+len_test, 0:inputdim].values
#y_test  = dataset.iloc[len_train:len_train+len_test, inputdim].values


In [7]:
len_x = int(len_data/nf)
ake_y  = y.reshape(len_x,nf)
ake_y=ake_y[:,0]
ake_y=np.divide(ake_y, 20).astype(int)
two_ch_input_x=X.reshape(len_x, nf ,inputdim)

In [8]:
len_train=int(len_x*0.9)
len_train=int(len_train/nf)*nf


len_test=int(len_x*0.1)
len_test=int(len_test/nf)*nf

print('len_train:', len_train)
print('len_test:', len_test)


('len_train:', 440335)
('len_test:', 48925)


In [9]:
del dataset

In [10]:
idx = np.random.permutation(len_x)
two_ch_input_x,ake_y = two_ch_input_x[idx], ake_y[idx]


In [11]:
two_ch_input_training = two_ch_input_x[0:len_train, :, :]
ake_y_train  = ake_y[0:len_train, ]
two_ch_input_test = two_ch_input_x[len_train:len_x, :, :]
ake_y_test  = ake_y[len_train:len_x, ]

In [12]:
two_ch_input_test.shape

(48929, 5, 2052)

In [13]:
def one_hot_encode(idx, vals=10):
    '''
    For use to one-hot encode the 10- possible labels
    '''
    len_idx=len(idx)
    out = np.zeros((len_idx, vals))
    for i in range(len_idx):
        out[i,idx[i]] = 1
        
    return out


In [14]:
two_ch_y_training = one_hot_encode(ake_y_train,10) 
two_ch_y_test = one_hot_encode(ake_y_test,10) 

In [15]:
# Data helper
class DataHelper():
    
    def __init__(self):
        self.i = 0
        self.k = 0
        
        #self.all_train_batches = [data_batch1,data_batch2,data_batch3,data_batch4,data_batch5]
        #self.test_batch = [test_batch]
        
        self.training_data = None
        self.training_labels = None
        
        self.test_data = None
        self.test_labels = None
    
    def set_up_data(self,two_ch_input_training,y_training,two_ch_input_test,y_test):
        
        print("Setting Up Training Data and Labels")
        
        self.training_data= two_ch_input_training 
        train_len = len(self.training_data)
        
        self.training_labels = y_training
        
        print("Setting Up Test Images and Labels")
        
        self.test_data = two_ch_input_test
        test_len = len(self.test_data)
        
        self.test_labels = y_test

        
    def next_batch(self, batch_size):
        x = self.training_data[self.i:self.i+batch_size]
        y = self.training_labels[self.i:self.i+batch_size]
        self.i = (self.i + batch_size) % len(self.training_data)
        return x, y
    
    def next_batch_test(self, batch_size):
        x_test = self.test_data[self.k:self.k+batch_size]
        y_test = self.test_labels[self.k:self.k+batch_size]
        #print(self.k)
        self.k = (self.k + batch_size) % len(self.test_data)
        
        return x_test, y_test

In [16]:
dh = DataHelper()
dh.set_up_data(two_ch_input_training,two_ch_y_training,two_ch_input_test,two_ch_y_test)

Setting Up Training Data and Labels
Setting Up Test Images and Labels


In [17]:

##### CREATING MODEL ########

import tensorflow as tf


with tf.name_scope("inputs"):
    x = tf.placeholder(tf.float32,[None,nf,inputdim],"x-input")
    x_image = tf.reshape(x, [-1, 2,image_row, image_col], name="x-image")  
    x_image =tf.transpose(x_image, perm=[0,2,3,1],name="x-imaget")  
    y_true = tf.placeholder(tf.float32,[None,10], "y-true")
    
#with tf.name_scope("hyperparameters"):
    #hold_prob = tf.constant(0.1, name='hold_prob')
    #hold_prob = tf.placeholder(tf.float32, name='hold_prob')
    


## HELPER FUNCTIONS ##

def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 1, 1, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b


In [18]:
## CREATE LAYERS ## 
    
convo_1 = convolutional_layer(x_image,shape=[2,2,2,64])
convo_1_pooling = max_pool_2by2(convo_1)

convo_2 = convolutional_layer(convo_1,shape=[2,2,64,64])
convo_2_pooling = max_pool_2by2(convo_2)

convo_3 = convolutional_layer(convo_2,shape=[2,2,64,64])

convo_2_flat = tf.reshape(convo_3,[-1,nf*2*513*64])

full_one_dropout = tf.nn.dropout(convo_2_flat,keep_prob=0.5)
full_layer_one = tf.nn.relu(normal_full_layer(full_one_dropout,512))

full_two_dropout = tf.nn.dropout(full_layer_one,keep_prob=0.5)

full_layer_two = tf.nn.relu(normal_full_layer(full_two_dropout,256))




y_pred = (normal_full_layer(full_layer_two,10))
y_pred = tf.identity(y_pred,'output_t');

## LOSS FUNCTION ##

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

## OPTIMIZER ##
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

init = tf.global_variables_initializer()


In [19]:
batch_size=20
size_test= int(len_test/batch_size)

print(size_test)

size_train= int(len_train/batch_size)
print(size_train)

2446
22016


In [20]:
epochnumber=20
checkpoint_dir = "models/"
tf.gfile.MakeDirs(checkpoint_dir)

with tf.Session() as sess:
    
    saver = tf.train.Saver()
    # tf.train.write_graph(sess.graph_def,
    #                     FLAGS.checkpoint_dir,
    #                    "graph.pbtxt",
    #                    True)
   
    sess.run(tf.global_variables_initializer())
    for epoch in range (epochnumber):
        train_batch_count = 0 
        train_batch_acc_total = 0
        for i in range(size_train):
            batch = dh.next_batch(batch_size)
            sess.run(train, feed_dict={x: batch[0], y_true: batch[1]})# hold_prob: 0.5
           


            if i%1000 == 0:
                print('Currently on training step {}'.format(i))


            # PRINT OUT A MESSAGE EVERY 100 STEPS
            if i%(size_train-1) == 0 and i!=0:
                print('Currently on step {} for Testing'.format(i))
                
                test_batch_acc_total = 0
                test_batch_count = 0
                matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
                acc = tf.reduce_mean(tf.cast(matches,tf.float32))
                

                for k in range (size_test):
                    batch_test = dh.next_batch_test(batch_size)
                    test_batch_acc_total += sess.run(acc,feed_dict={x:batch_test[0],y_true:batch_test[1]}) #,hold_prob:1.0
                    test_batch_count += 1
                    train_batch_acc_total += sess.run(acc,feed_dict={x:batch[0],y_true:batch[1]}) #,hold_prob:1.0
                    train_batch_count += 1


                    if k%1000 == 0:
                        print('Currently on testing step {}'.format(k))

                print('Epoch: {}'.format(epoch))
                print('Testing Accuracy: {}\n'.format(test_batch_acc_total/test_batch_count))
                print('Training Accuracy: {}\n'.format(train_batch_acc_total/train_batch_count))


    tf.train.write_graph(sess.graph_def, checkpoint_dir, 'graph_DOA_2.pbtxt', as_text=True)
#    tf.train.Saver(tf.trainable_variables()).save(sess, 'Babble_0_Real_Imag_Dr40/my-model-chkpt')
    tf.gfile.MakeDirs( checkpoint_dir + str(epoch))
    checkpoint_file = os.path.join(  checkpoint_dir + str(epoch), "model")
    saver.save(sess, checkpoint_file)
    print("**** SAVED MODEL ****")      
    print("**** COMPLETED ALL THE EPOCHS ****")



Currently on training step 0
Currently on training step 1000
Currently on training step 2000
Currently on training step 3000
Currently on training step 4000
Currently on training step 5000
Currently on training step 6000
Currently on training step 7000
Currently on training step 8000
Currently on training step 9000
Currently on training step 10000
Currently on training step 11000
Currently on training step 12000
Currently on training step 13000
Currently on training step 14000
Currently on training step 15000
Currently on training step 16000
Currently on training step 17000
Currently on training step 18000
Currently on training step 19000
Currently on training step 20000
Currently on training step 21000
Currently on training step 22000
Currently on step 22015 for Testing
Currently on testing step 0
Currently on testing step 1000
Currently on testing step 2000
Epoch: 0
Testing Accuracy: 0.865739983169

Training Accuracy: 0.87820931945

Currently on training step 0
Currently on training 

Currently on training step 19000
Currently on training step 20000
Currently on training step 21000
Currently on training step 22000
Currently on step 22015 for Testing
Currently on testing step 0
Currently on testing step 1000
Currently on testing step 2000
Epoch: 8
Testing Accuracy: 0.950640493914

Training Accuracy: 0.951124276207

Currently on training step 0
Currently on training step 1000
Currently on training step 2000
Currently on training step 3000
Currently on training step 4000
Currently on training step 5000
Currently on training step 6000
Currently on training step 7000
Currently on training step 8000
Currently on training step 9000
Currently on training step 10000
Currently on training step 11000
Currently on training step 12000
Currently on training step 13000
Currently on training step 14000
Currently on training step 15000
Currently on training step 16000
Currently on training step 17000
Currently on training step 18000
Currently on training step 19000
Currently on trai

Currently on training step 8000
Currently on training step 9000
Currently on training step 10000
Currently on training step 11000
Currently on training step 12000
Currently on training step 13000
Currently on training step 14000
Currently on training step 15000
Currently on training step 16000
Currently on training step 17000
Currently on training step 18000
Currently on training step 19000
Currently on training step 20000
Currently on training step 21000
Currently on training step 22000
Currently on step 22015 for Testing
Currently on testing step 0
Currently on testing step 1000
Currently on testing step 2000
Epoch: 17
Testing Accuracy: 0.957236297118

Training Accuracy: 0.966373663722

Currently on training step 0
Currently on training step 1000
Currently on training step 2000
Currently on training step 3000
Currently on training step 4000
Currently on training step 5000
Currently on training step 6000
Currently on training step 7000
Currently on training step 8000
Currently on trai